In [2]:
import pandas as pd
metadata = pd.read_csv('/home/mali/RecommendationEngine/data/external/movies_metadata.csv',low_memory=False)

In [6]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [25]:
metadata.shape

(45466, 24)

WeightedRating(WR)=((v/v+m)⋅R)+((m/v+m)⋅C)

In the above equation,

    v is the number of votes for the movie;

    m is the minimum votes required to be listed in the chart;

    R is the average rating of the movie;

    C is the mean vote across the whole report.


In [8]:
C = metadata['vote_average'].mean()
print(C)

5.618207215134185


In [9]:
m = metadata['vote_count'].quantile(.9)
print(m)

160.0


In [27]:
q_movies = metadata.copy().loc[metadata['vote_count']>=m]
q_movies.shape

(4555, 24)

In [43]:
def weighted_average(x,m=m,C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (v + m) * C)

In [44]:
q_movies['score'] = q_movies.apply(weighted_average,axis=1)

In [45]:
q_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
10309,False,NaN,13200000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,19404,tt0112870,hi,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",...,100000000.0,190.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Come... Fall In Love,Dilwale Dulhania Le Jayenge,False,9.1,661.0,8.421453
39085,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,192040,tt0795176,en,Planet Earth,A documentary miniseries described by its make...,...,0.0,550.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,planet earth as you've never seen it before,Planet Earth,False,8.8,176.0,7.284861
40251,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",https://www.funimationfilms.com/movie/yourname/,372058,tt5311514,ja,君の名は。,High schoolers Mitsuha and Taki are complete s...,...,355298270.0,106.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Your Name.,False,8.5,1030.0,8.112532
834,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,245066411.0,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0,8.425439
314,False,NaN,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,28341469.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358.0,8.445869


In [46]:
q_movies = q_movies.sort_values('score',ascending=False)
q_movies[['title','vote_average','vote_count','score']].head(15)

,title,vote_average,vote_count,score
314,The Shawshank Redemption,8.5,8358.0,8.445869
834,The Godfather,8.5,6024.0,8.425439
10309,Dilwale Dulhania Le Jayenge,9.1,661.0,8.421453
12481,The Dark Knight,8.3,12269.0,8.265477
2843,Fight Club,8.3,9678.0,8.256385
292,Pulp Fiction,8.3,8670.0,8.251406
522,Schindler's List,8.3,4436.0,8.206639
23673,Whiplash,8.3,4376.0,8.205404
5481,Spirited Away,8.3,3968.0,8.196055
2211,Life Is Beautiful,8.3,3643.0,8.187171


CONTENT-BASED RECOMMENDER

In [47]:
metadata['overview'].head(3)

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
Name: overview, dtype: object

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
metadata['overview'] = metadata['overview'].fillna('')

In [52]:
tfidf_matrix = tfidf.fit_transform(metadata['overview'])
tfidf_matrix.shape

(45466, 75827)

In [60]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [61]:
cosine_sim.shape

(45466, 45466)

In [69]:
indices = pd.Series(metadata.index, index = metadata['title']).drop_duplicates()
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [76]:
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = indices[title]
    #print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print(sim_scores)
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return metadata['title'].loc[movie_indices]

In [79]:
get_recommendations('The Godfather')

1178               The Godfather: Part II
44030    The Godfather Trilogy: 1972-1990
1914              The Godfather: Part III
23126                          Blood Ties
11297                    Household Saints
34717                   Start Liquidation
10821                            Election
38030            A Mother Should Be Loved
17729                   Short Sharp Shock
26293                  Beck 28 - Familjen
Name: title, dtype: object